# Statistics

In Smile, there are many statistical functions to describe and analyze data. For example, use the following functions to calculate the descriptive statistics for your data: `sum`, `mean`, `median`, `q1`, `q3`, `variance`, `sd`, `mad` (median absolute deviation), `min`, `max`, `whichMin`, `whichMax`, etc.

In [ ]:
import $ivy.`com.github.haifengl::smile-scala:2.2.2`
import $ivy.`org.slf4j:slf4j-simple:1.7.30` 

import java.lang.Math._
import smile.math._
import smile.math.MathEx._
import smile.stat._
import smile.stat.distribution._

In [ ]:
mean(Array(1.0, 2.0, 3.0, 4.0))

sd(Array(1.0, 2.0, 3.0, 4.0))

## Distributions

Probability distributions are theoretical distributions based on assumptions about a source population. The distributions assign probability to the event that a random variable has a specific, discrete value, or falls within a specified range of continuous values.

All univariate distributions in Smile implements the interface `smile.math.stat.distribution.Distribution`. We support Bernoulli, beta, binomial, χ2, exponential, F, gamma, Gaussian, geometric, hyper geometric, logistic, log normal, negative binomial, Possion, shift geometric, t, and Weibull distribution. In additional, multivariate Gaussian distribution is supported. In fact, we also support finite mixture models and can estimate the exponential family mixture models from data.

A `Distribution` object can be created with given parameters. Meanwhile they can be created by estimating parameters from a given data set. With a `Distribution` object, we may access its distribution parameter(s), mean, variance, standard deviation, entropy, generates a random number following the distribution, call its probability density function (the method p or cumulative distribution function (cdf). The reverse function of cdf is quantile. We can also calculate the likelihood or log likelihood of a sample set.

In [ ]:
val e = new ExponentialDistribution(1.0)

e.mean

e.variance

e.sd

e.entropy

// generate a random number
e.rand

// PDF
e.p(2.0)

e.cdf(2.0)

e.quantile(0.1)

e.logLikelihood(Array(1.0, 1.1, 0.9, 1.5))

The below example estimates a distribution from data.

In [ ]:
val e = ExponentialDistribution.fit(Array(1.0, 1.1, 0.9, 1.5, 1.8, 1.9, 2.0, 0.5))

The below is a more advanced example of estimating a mixture model of Gaussian, exponential and gamma distribution. The result is quite accurate for this complicated case.

In [ ]:
val gaussian = new GaussianDistribution(-2.0, 1.0)
val exp = new ExponentialDistribution(0.8)
val gamma = new GammaDistribution(2.0, 3.0)

// generate the samples
val data = Array.fill(500)(gaussian.rand()) ++ Array.fill(500)(exp.rand()) ++ Array.fill(1000)(gamma.rand())

// define the initial guess of the components in the mixture model
val a = new Mixture.Component(0.3, new GaussianDistribution(0.0, 1.0))
val b = new Mixture.Component(0.3, new ExponentialDistribution(1.0))
val c = new Mixture.Component(0.4, new GammaDistribution(1.0, 2.0))

// estimate the model
val mixture = ExponentialFamilyMixture.fit(data, a, b, c)

If the distribution family is not known, nonparametric methods such as kernel density estimation can be used. Kernel density estimation is a fundamental data smoothing problem where inferences about the population are made, based on a finite data sample. It is also known as the Parzen window method.

In [ ]:
val k = new KernelDensity(data)

k.p(1.0)

mixture.p(1.0)

## Hypothesis Test

A statistical hypothesis test is a method of making decisions using data, whether from a controlled experiment or an observational study (not controlled). In statistics, a result is called statistically significant if it is unlikely to have occurred by chance alone, according to a pre-determined threshold probability, the significance level.

### χ2 Test

#### One-Sample Test

Given the array x containing the observed numbers of events, and an array prob containing the expected probabilities of events, and given the number of constraints (normally one), a small value of p-value indicates a significant difference between the distributions.

In [ ]:
val bins = Array(20, 22, 13, 22, 10, 13)
val prob = Array(1.0/6, 1.0/6, 1.0/6, 1.0/6, 1.0/6, 1.0/6)
chisqtest(bins, prob)

#### Two-Sample Test

Given the arrays x and y, containing two sets of binned data, and given one constraint, a small value of p-value indicates a significant difference between two distributions.

In [ ]:
val bins1 = Array(8, 13, 16, 10, 3)
val bins2 = Array(4,  9, 14, 16, 7)
chisqtest2(bins1, bins2)

#### Independence Test

Independence test on a two-dimensional contingency table in the form of an array of integers. The rows of contingency table are labels by the values of one nominal variable, the columns are labels by the values of the other nominal variable, and whose entries are non-negative integers giving the number of observed events for each combination of row and column. Continuity correction will be applied when computing the test statistic for 2x2 tables: one half is subtracted from all |O-E| differences. The correlation coefficient is calculated as Cramer's V.

In [ ]:
val x = Array(Array(12, 7), Array(5, 7))
chisqtest(x)

### F Test

Test if the arrays x and y have significantly different variances. Small values of p-value indicate that the two arrays have significantly different variances.

In [ ]:
val x = Array(0.48074284, -0.52975023, 1.28590721, 0.63456079, -0.41761197, 2.76072411,
            1.30321095, -1.16454533, 2.27210509, 1.46394553, -0.31713164, 1.26247543,
            2.65886430, 0.40773450, 1.18055440, -0.39611251, 2.13557687, 0.40878860,
            1.28461394, -0.02906355)

val y = Array(1.7495879, 1.9359727, 3.1294928, 0.0861894, 2.1643415, 0.1913219,
            -0.3947444, 1.6910837, 1.1548294, 0.2763955, 0.4794719, 3.1805501,
            1.5700497, 2.6860190, -0.4410879, 1.8900183, 1.3422381, -0.1701592)

ftest(x, y)

val z = Array(0.6621329, 0.4688975, -0.1553013, 0.4564548, 2.2776146, 2.1543678,
            2.8555142, 1.5852899, 0.9091290, 1.6060025, 1.0111968, 1.2479493,
            0.9407034, 1.7167572, 0.5380608, 2.1290007, 1.8695506, 1.2139096)

ftest(x, z)

### t Test

#### One-Sample Test
Independent one-sample t-test whether the mean of a normally distributed population has a value specified in a null hypothesis. Small values of p-value indicate that the array has significantly different mean.

In [ ]:
val x = Array(0.48074284, -0.52975023, 1.28590721, 0.63456079, -0.41761197, 2.76072411,
            1.30321095, -1.16454533, 2.27210509, 1.46394553, -0.31713164, 1.26247543,
            2.65886430, 0.40773450, 1.18055440, -0.39611251, 2.13557687, 0.40878860,
            1.28461394, -0.02906355)
        
ttest(x, 1.0)

ttest(x, 1.1)

#### Paired Two-Sample Test
Given the paired arrays x and y, test if they have significantly different means. Small values of p-value indicate that the two arrays have significantly different means.

In [ ]:
val y = Array(1.7495879, 1.9359727, 3.1294928, 0.0861894, 2.1643415, 0.1913219,
            -0.3947444, 1.6910837, 1.1548294, 0.2763955, 0.4794719, 3.1805501,
            1.5700497, 2.6860190, -0.4410879, 1.8900183, 1.3422381, -0.1701592)

val z = Array(0.6621329, 0.4688975, -0.1553013, 0.4564548, 2.2776146, 2.1543678,
            2.8555142, 1.5852899, 0.9091290, 1.6060025, 1.0111968, 1.2479493,
            0.9407034, 1.7167572, 0.5380608, 2.1290007, 1.8695506, 1.2139096)

ttest(y, z)

#### Independent (Unpaired) Two-Sample Test
Test if the arrays x and y have significantly different means. Small values of p-value indicate that the two arrays have significantly different means. If the parameter equalVariance is true, the data arrays are assumed to be drawn from populations with the same true variance. Otherwise, The data arrays are allowed to be drawn from populations with unequal variances.

In [ ]:
val x = Array(0.48074284, -0.52975023, 1.28590721, 0.63456079, -0.41761197, 2.76072411,
            1.30321095, -1.16454533, 2.27210509, 1.46394553, -0.31713164, 1.26247543,
            2.65886430, 0.40773450, 1.18055440, -0.39611251, 2.13557687, 0.40878860,
            1.28461394, -0.02906355)

val y = Array(1.7495879, 1.9359727, 3.1294928, 0.0861894, 2.1643415, 0.1913219,
            -0.3947444, 1.6910837, 1.1548294, 0.2763955, 0.4794719, 3.1805501,
            1.5700497, 2.6860190, -0.4410879, 1.8900183, 1.3422381, -0.1701592)

ttest2(x, y)
ttest2(x, y, true)

val z = Array(0.6621329, 0.4688975, -0.1553013, 0.4564548, 2.2776146, 2.1543678,
            2.8555142, 1.5852899, 0.9091290, 1.6060025, 1.0111968, 1.2479493,
            0.9407034, 1.7167572, 0.5380608, 2.1290007, 1.8695506, 1.2139096)

ttest2(x, z)
ttest2(x, z, true)

### Kolmogorov–Smirnov Test
#### One-Sample Test
The one-sample K-S test for the null hypothesis that the data set x is drawn from the given distribution. Small values of p-value show that the cumulative distribution function of x is significantly different from the given distribution. The array x is modified by being sorted into ascending order.

In [ ]:
val x = Array(
            0.53236606, -1.36750258, -1.47239199, -0.12517888, -1.24040594, 1.90357309,
            -0.54429527, 2.22084140, -1.17209146, -0.68824211, -1.75068914, 0.48505896,
            2.75342248, -0.90675303, -1.05971929, 0.49922388, -1.23214498, 0.79284888,
            0.85309580, 0.17903487, 0.39894754, -0.52744720, 0.08516943, -1.93817962,
            0.25042913, -0.56311389, -1.08608388, 0.11912253, 2.87961007, -0.72674865,
            1.11510699, 0.39970074, 0.50060532, -0.82531807, 0.14715616, -0.96133601,
            -0.95699473, -0.71471097, -0.50443258, 0.31690224, 0.04325009, 0.85316056,
            0.83602606, 1.46678847, 0.46891827, 0.69968175, 0.97864326, 0.66985742,
            -0.20922486, -0.15265994)

kstest(x, new GaussianDistribution(0, 1))

#### Two-Sample Test
The two-sample K–S for the null hypothesis that the data sets are drawn from the same distribution. Small values of p-value show that the cumulative distribution function of x is significantly different from that of y. The arrays x and y are modified by being sorted into ascending order.

In [ ]:
val x = Array(
            0.53236606, -1.36750258, -1.47239199, -0.12517888, -1.24040594, 1.90357309,
            -0.54429527, 2.22084140, -1.17209146, -0.68824211, -1.75068914, 0.48505896,
            2.75342248, -0.90675303, -1.05971929, 0.49922388, -1.23214498, 0.79284888,
            0.85309580, 0.17903487, 0.39894754, -0.52744720, 0.08516943, -1.93817962,
            0.25042913, -0.56311389, -1.08608388, 0.11912253, 2.87961007, -0.72674865,
            1.11510699, 0.39970074, 0.50060532, -0.82531807, 0.14715616, -0.96133601,
            -0.95699473, -0.71471097, -0.50443258, 0.31690224, 0.04325009, 0.85316056,
            0.83602606, 1.46678847, 0.46891827, 0.69968175, 0.97864326, 0.66985742,
            -0.20922486, -0.15265994)
val y = Array(
            0.95791391, 0.16203847, 0.56622013, 0.39252941, 0.99126354, 0.65639108,
            0.07903248, 0.84124582, 0.76718719, 0.80756577, 0.12263981, 0.84733360,
            0.85190907, 0.77896244, 0.84915723, 0.78225903, 0.95788055, 0.01849366,
            0.21000365, 0.97951772, 0.60078520, 0.80534223, 0.77144013, 0.28495121,
            0.41300867, 0.51547517, 0.78775718, 0.07564151, 0.82871088, 0.83988694)

kstest(x, y)

### Correlation Test
#### Pearson Correlation
The t-test is used to establish if the correlation coefficient is significantly different from zero, and, hence that there is evidence of an association between the two variables. There is then the underlying assumption that the data is from a normal distribution sampled randomly. If this is not true, then it is better to use Spearman's coefficient of rank correlation (for non-parametric variables).

In [ ]:
val x = Array(44.4, 45.9, 41.9, 53.3, 44.7, 44.1, 50.7, 45.2, 60.1)
val y  = Array(2.6,  3.1,  2.5,  5.0,  3.6,  4.0,  5.2,  2.8,  3.8)

pearsontest(x, y)

#### Spearman Rank Correlation
The Spearman Rank Correlation Coefficient is a form of the Pearson coefficient with the data converted to rankings (i.e. when variables are ordinal). It can be used when there is non-parametric data and hence Pearson cannot be used.

The raw scores are converted to ranks and the differences between the ranks of each observation on the two variables are calculated.

The p-value is calculated by approximation, which is good for n > 10.

In [ ]:
spearmantest(x, y)

#### Kendall Rank Correlation
The Kendall Tau Rank Correlation Coefficient is used to measure the degree of correspondence between sets of rankings where the measures are not equidistant. It is used with non-parametric data. The p-value is calculated by approximation, which is good for n > 10.

In [ ]:
kendalltest(x, y)